In [2]:
import csv
import random

def is_winner(board, player):
    # Check rows, columns, and diagonals
    for i in range(3):
        if all(board[i][j] == player for j in range(3)) or \
           all(board[j][i] == player for j in range(3)):
            return True
    if all(board[i][i] == player for i in range(3)) or \
       all(board[i][2-i] == player for i in range(3)):
        return True
    return False

def get_empty_cells(board):
    return [(i, j) for i in range(3) for j in range(3) if board[i][j] == ' ']

def generate_tictactoe_logs(num_games, output_file):
    with open(output_file, 'w', newline='') as csvfile:
        fieldnames = ['event_type', 'agent_id', 'context']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for _ in range(num_games):
            board = [[' ' for _ in range(3)] for _ in range(3)]
            
            # Game Start
            writer.writerow({
                'event_type': 'GAME_START',
                'agent_id': 'system',
                'context': 'New Game'
            })

            current_player = 'X'
            game_result = None

            while True:
                empty_cells = get_empty_cells(board)
                if not empty_cells:
                    game_result = 'draw'
                    break

                move = random.choice(empty_cells)
                board[move[0]][move[1]] = current_player

                writer.writerow({
                    'event_type': 'MOVE',
                    'agent_id': current_player,
                    'context': f"{move[0]},{move[1]}"
                })

                if is_winner(board, current_player):
                    game_result = current_player
                    break

                current_player = 'O' if current_player == 'X' else 'X'

            # Game End
            writer.writerow({
                'event_type': 'GAME_END',
                'agent_id': 'system',
                'context': game_result
            })

    print(f"Generated logs for {num_games} Tic Tac Toe games. Saved to {output_file}")

In [3]:
generate_tictactoe_logs(num_games=1000, output_file='../../../processed/games/tic-tac-toe/1k_single_agent.csv')

Generated logs for 1000 Tic Tac Toe games. Saved to ../../../processed/games/tic-tac-toe/1k_single_agent.csv


In [4]:
generate_tictactoe_logs(num_games=1000 * 10, output_file='../../../processed/games/tic-tac-toe/10k_single_agent.csv')

Generated logs for 10000 Tic Tac Toe games. Saved to ../../../processed/games/tic-tac-toe/10k_single_agent.csv


In [5]:
generate_tictactoe_logs(num_games=1000 * 50, output_file='../../../processed/games/tic-tac-toe/50k_single_agent.csv')

Generated logs for 50000 Tic Tac Toe games. Saved to ../../../processed/games/tic-tac-toe/50k_single_agent.csv


In [1]:
import csv
import random

def is_winner(board, player):
    # Check rows, columns, and diagonals
    for i in range(3):
        if all(board[i][j] == player for j in range(3)) or \
           all(board[j][i] == player for j in range(3)):
            return True
    if all(board[i][i] == player for i in range(3)) or \
       all(board[i][2-i] == player for i in range(3)):
        return True
    return False

def get_empty_cells(board):
    return [(i, j) for i in range(3) for j in range(3) if board[i][j] == ' ']

def minimax(board, depth, is_maximizing):
    if is_winner(board, 'X'):
        return 10 - depth
    if is_winner(board, 'O'):
        return depth - 10
    if not get_empty_cells(board):
        return 0

    if is_maximizing:
        best_score = -float('inf')
        for (i, j) in get_empty_cells(board):
            board[i][j] = 'X'
            score = minimax(board, depth + 1, False)
            board[i][j] = ' '
            best_score = max(score, best_score)
        return best_score
    else:
        best_score = float('inf')
        for (i, j) in get_empty_cells(board):
            board[i][j] = 'O'
            score = minimax(board, depth + 1, True)
            board[i][j] = ' '
            best_score = min(score, best_score)
        return best_score

def find_best_move(board, player):
    best_move = None
    best_score = -float('inf') if player == 'X' else float('inf')

    for (i, j) in get_empty_cells(board):
        board[i][j] = player
        score = minimax(board, 0, player == 'O')
        board[i][j] = ' '
        if (player == 'X' and score > best_score) or (player == 'O' and score < best_score):
            best_score = score
            best_move = (i, j)
    
    return best_move

def generate_tictactoe_logs(num_games, output_file):
    with open(output_file, 'w', newline='') as csvfile:
        fieldnames = ['event_type', 'agent_id', 'context']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for _ in range(num_games):
            board = [[' ' for _ in range(3)] for _ in range(3)]
            
            # Game Start
            writer.writerow({
                'event_type': 'GAME_START',
                'agent_id': 'system',
                'context': 'New Game'
            })

            current_player = 'X'
            game_result = None

            while True:
                empty_cells = get_empty_cells(board)
                if not empty_cells:
                    game_result = 'draw'
                    break

                move = find_best_move(board, current_player)
                if move:
                    board[move[0]][move[1]] = current_player

                    writer.writerow({
                        'event_type': 'MOVE',
                        'agent_id': current_player,
                        'context': f"{move[0]},{move[1]}"
                    })

                    if is_winner(board, current_player):
                        game_result = current_player
                        break

                    current_player = 'O' if current_player == 'X' else 'X'
                else:
                    break  # No valid moves left

            # Game End
            writer.writerow({
                'event_type': 'GAME_END',
                'agent_id': 'system',
                'context': game_result
            })

    print(f"Generated logs for {num_games} Tic Tac Toe games. Saved to {output_file}")

# Example usage
generate_tictactoe_logs(10, 'tictactoe_logs.csv')


Generated logs for 10 Tic Tac Toe games. Saved to tictactoe_logs.csv


In [2]:
generate_tictactoe_logs(num_games=1000, output_file='../../../processed/games/tic-tac-toe/1k_single_agent_minmax.csv')

KeyboardInterrupt: 